In [112]:
import json

with open("../datasets/etel_adnan.json", "r") as f:
    data = f.read()
    data = json.loads(data)

In [113]:
data[0][0]

{'role': 'LA',
 'content': 'Etel, you are a writer, a poet, an artist; you were born in Lebanon. In which language were you brought up?'}

In [114]:
from datasets import Dataset

ds = Dataset.from_dict({"messages": data})

In [115]:
from transformers import AutoTokenizer

model_name = "microsoft/phi-4"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [116]:
tokenizer.chat_template = "{% for message in messages %}{% if (message['role'] == 'system') %}{{'<|im_start|>system<|im_sep|>' + message['content'] + '<|im_end|>'}}{% elif (message['role'] == 'LA') %}{{'<|im_start|>Interviewer<|im_sep|>' + message['content'] + '<|im_end|>'}}{% elif (message['role'] == 'EA') %}{{'<|im_start|>Etel Adnan<|im_sep|>' + message['content'] + '<|im_end|>'}}{% elif (message['role'] == 'user') %}{{'<|im_start|>user<|im_sep|>' + message['content'] + '<|im_end|>'}}{% elif (message['role'] == 'assistant') %}{{'<|im_start|>assistant<|im_sep|>' + message['content'] + '<|im_end|>'}}{% endif %}{% endfor %}"

In [121]:
def template_tokenize_chunk(example):
    templated_tokenized = tokenizer.apply_chat_template(
        example["messages"], tokenize=False,
    )
    return {"templated_messages": templated_tokenized}

In [122]:
templated_ds = ds.map(lambda x: template_tokenize_chunk(x), batched=False)

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [123]:
templated_ds

Dataset({
    features: ['messages', 'templated_messages'],
    num_rows: 11
})

In [133]:
from datasets import load_dataset
ts_ds = load_dataset("karpathy/tiny_shakespeare", trust_remote_code=True)

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [135]:
ts_ds_tokenized= ts_ds.map(
    lambda x: tokenizer(
        x["text"],
        max_length=128,
        truncation=True,
        padding="max_length",
    ),
    batched=True,
)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [143]:
len(ts_ds_tokenized["train"]["input_ids"])

1

In [129]:
result=tokenizer(
    templated_ds["templated_messages"],
    max_length=128,
    truncation=True,
    padding="max_length",
)

In [131]:
len(result["input_ids"])

11

In [124]:
tokenized_ds = templated_ds.map(
    lambda x: tokenizer(
        x["templated_messages"],
        max_length=128,
        truncation=True,
        padding="max_length",
    )
)

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [125]:
tokenized_ds

Dataset({
    features: ['messages', 'templated_messages', 'input_ids', 'attention_mask'],
    num_rows: 11
})

In [126]:
tokenized_ds[0]["input_ids"][:3]

[100264, 62434, 261]

In [127]:
len(tokenized_ds[0]["input_ids"])

128

In [82]:
def chunk_input_ids(example, chunk_size=128):
    input_ids = example["input_ids"]
    chunks = [
        input_ids[i : i + chunk_size] for i in range(0, len(input_ids), chunk_size)
    ]
    return chunks


# Apply the chunking function
chunk_size = 128
chunked_ds = tokenized_ds.map(
    lambda example: {"chunks": chunk_input_ids(example, chunk_size=chunk_size)},
    batched=False,
    remove_columns=ds.column_names,
)

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [85]:
chunked_ds = chunked_ds.flatten()

In [86]:
chunked_ds

Dataset({
    features: ['chunks'],
    num_rows: 11
})

In [51]:
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)


In [57]:
collator(ds[0])

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`messages` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [36]:
ds = ds.flatten()

In [37]:
ds

Dataset({
    features: ['messages', 'input_ids'],
    num_rows: 11
})